# Finding Planted Clique in Hypergraphs

Efficient recovery of planted cliques in hypergraphs.
A model on $n$ vertices with order $d$.
$K$ vertices forming a clique, meaning that any size-d tuple within the clique is connected by a hyperedge.
All other size-$d$ tuples form a hyperedge with probability $q = 1/2$ .

In [227]:
from itertools import permutations
import math
import time as tm

import numpy as np

In [228]:
# set the degree of each vertex on graph to 1
def add_all(i, j, k, g):
    g[np.asarray(list(set(permutations([i,j,k]))))] = 1
    # g[i, j, k] = 1
    # g[i, k, j] = 1
    # g[j, i, k] = 1
    # g[j, k, i] = 1
    # g[k, i, j] = 1
    # g[k, j, i] = 1
    return g

In [229]:
# set the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[np.asarray(list(set(permutations([i,j,k]))))] = 0
    # g[i, j, k] = 0
    # g[i, k, j] = 0
    # g[j, i, k] = 0
    # g[j, k, i] = 0
    # g[k, i, j] = 0
    # g[k, j, i] = 0
    return g


In [230]:
# # improvement
# ll = [1,2,0]
# # ind = np.asarray(list(set(permutations(ll))))
# # ind
# #
# g = np.zeros((4,4,4))
# # g[ind] = 1
# # g
# g[np.asarray(list(set(permutations(ll))))] = 1
# g

In [231]:
### create a graph with planted clique

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clique_size: clique size

def generate_graph(num_vertices, pr, clique_size):
    def plant_clique():
        # print('\nHERE\n')
        for ii in range(clique_size):
            for jj in range(ii + 1, clique_size):
                for kk in range(jj + 1, clique_size):
                    # aa = clique_v[ii]
                    # print('\n******',aa, vec[aa])
                    # vec[clique_v[ii]] += 1
                    # bb = clique_v[jj]
                    # print('\n******',bb, vec[bb] )
                    # vec[clique_v[jj]] += 1
                    # cc = clique_v[kk]
                    # print('\n******',cc, vec[cc])
                    # vec[clique_v[kk]] += 1
                    idx_list = [clique_v[ii],clique_v[jj],clique_v[kk]]
                    vec = [vec[idx] + 1 for idx in idx_list]
                    add_all(clique_v[ii], clique_v[jj], clique_v[kk], g)

    # g = np.array([np.array([np.array([0 for _ in range(0, num_vertices)]) for _ in range(num_vertices)]) for _ in
    #               range(num_vertices)])
    g = np.zeros((num_vertices, num_vertices, num_vertices))
    # print(f"\n******** {g} ********\n")
    # vec = np.array([0 for _ in range(0, num_vertices)])
    vec = np.zeros(num_vertices,)
    # print(f"\n******** {vec} ********\n")

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    clique_v = np.random.choice(range(num_vertices), clique_size, replace=False)
    print(f"\n******** {clique_v} ********\n")

    plant_clique()

    return g, vec, clique_v


In [232]:
# calculate the total number of edges on a graph with num_vertices vertices
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [233]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    # active_count: number of vertices that are associated with at least 1 edge
    active_count = np.count_nonzero(vec)
    # number of edges
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == math.comb(active_count, 3):
        return True
    return False

In [234]:
# remove edge
def remove_edges(g, num_vertices, vec, curr_idx):
    vec[curr_idx] = 0
    graph_copy = g.copy()
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if graph_copy[curr_idx, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr_idx, j, k, graph_copy)
    return graph_copy, vec

### Generate a graph
n: number of vertices
p: probability of an edge being included
k: clique size

In [235]:
# driver program

N = 5  # number of vertices
P = 0.5  # probability of an edge being included
K = 3  # clique size

# generate graph
start_generate = tm.time()
res = generate_graph(N, P, K)

# G graph with planted clique
# V: vector storing the number of edges associated with each vertex in the graph
# clique_vertices: the set of clique vertices
G, V, clique_vertices = res[0], res[1], res[-1]

# print(f"graph: {G}\n")
G_0 = G.copy()
print(f"edge-occurrence vector: {V}\n")
print(f"set of clique vertices after removal phase: {clique_vertices}\n")

time_generate = np.round(tm.time() - start_generate, 3)
# print("Time taken to generate graph: ", "seconds")



******** [1 4 2] ********


HERE

edge-occurrence vector: [3. 4. 3. 3. 5.]

set of clique vertices after removal phase: [1 4 2]



### Removal Phase

In [236]:
start_removal = tm.time()
itr = 0
removed = []  # keep track of the vertices removed from the original graph to form a clique

# clique = is_clique(G, V, N)

while not is_clique(G, V, N):

    itr += 1

    curr = -1
    idx_sorted = np.argsort(V)

    print(f"i = {itr}\nindex sorted: {idx_sorted}\n")
    # print(f"number of edges associated with each vertex: {V}")

    for idx in range(N):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break
    # print(f"vertex removed: {curr} number of edges: {V[curr]}\n")
    A = remove_edges(G, N, V, curr)
    G = A[0]

    clique = is_clique(G, V, N)

print(f"is a clique at iteration #{itr}!!!")
time_remove = np.round(tm.time() - start_removal, 3)

is a clique at iteration #0!!!


In [237]:
print(f"number of iterations in the removal phase: {itr}")

number of iterations in the removal phase: 0


In [238]:
full_vertices = np.arange(N)
included = np.setdiff1d(full_vertices, removed)
assert len(removed) + len(included) == N
print(f"vertices included after the removal phase: {included}")

vertices included after the removal phase: [0 1 2 3 4]


### Inclusion Phase

In [239]:
def inclusion_phase(target, in_set, g):
    def connected():
        for j in range(len(in_set)):
            for k in range(j + 1, len(in_set)):
                if g[target_idx, in_set[j], in_set[k]] != 1:
                    # print(f" vertex {idx} is not connected to all clique vertices")
                    return False
        return True

    for target_idx in target:
        if connected():
            print(f"add {target_idx} to clique!")
            in_set = np.append(in_set, idx)

    return in_set


In [240]:
start_include = tm.time()
res = inclusion_phase(removed, included, G_0)
time_include = np.round(tm.time() - start_include, 3)

In [241]:
print(f"Set of clique vertices after generation phase: {np.sort(clique_vertices)}\n")
print(f"Set of clique vertices after removal phase: {np.sort(included)}\n")
# print(f"set of vertices removed: {removed}, number of elements = {len(removed)}\n")
print(f"Set of clique vertices after inclusion phase: {np.sort(res)}")

Set of clique vertices after generation phase: [1 2 4]

Set of clique vertices after removal phase: [0 1 2 3 4]

Set of clique vertices after inclusion phase: [0 1 2 3 4]
